In [3]:
import matplotlib.pyplot as plt 
import scipy.misc
import json

import numpy as np
from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, Cropping2D
from keras.layers import MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation, Add, merge, concatenate
from keras.models import Model
from keras.utils.layer_utils import print_summary
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from scipy.misc import imread

from keras import __version__ as kv
from scipy import __version__ as sv
from matplotlib import __version__ as mv
from numpy import __version__ as nv

print kv
print sv
print mv
print nv

Using TensorFlow backend.


2.0.5
0.19.1
2.0.0
1.13.0


In [4]:
# Local files
import utils
reload(utils)

<module 'utils' from 'utils.pyc'>

# Binary turbine classifier, with distance-to-land feature

In [3]:
trained_model_dir = '/home/ubuntu/data/sar/experiment_crops_20170815/trained_models/5.0-as-binary_classifier/'
train_dir = '/home/ubuntu/data/sar/experiment_crops_20170815/binary_experiments/turbine_classifier/train/50x50'
valid_dir = '/home/ubuntu/data/sar/experiment_crops_20170815/binary_experiments/turbine_classifier/validate/50x50/'


In [ ]:
def add_dist2land_experiment_crops_20170815_turbine():
    
    train_dir = '/home/ubuntu/data/sar/experiment_crops_20170815/binary_experiments/turbine_classifier/train/50x50/'
    valid_dir = '/home/ubuntu/data/sar/experiment_crops_20170815/binary_experiments/turbine_classifier/validate/50x50/'

    train_class = []           
    train_filename = []
    train_crops = []
    train_feature = []

    valid_class = []
    valid_filename = []
    valid_crops = []
    valid_feature = []

    train_class_desc = 'other'
    train_class_array = [0]
    with open('/home/ubuntu/data/sar/experiment_crops_20170815/train/distance_to_land/experiments_train_oil_and_gas_infrastructure.json') as json_data:
        json_train_data = json.load(json_data)
        for id_, item in json_train_data.iteritems():
            fn = id_.replace('.tif', '.png')
            train_filename.append(fn)
            train_feature.append(item['distance to land'])
            train_class.append(train_class_array)
            file_path = train_dir + '/' + train_class_desc + '/' + fn
            try:
                img  = imread(file_path)
            except IOError:
                continue
            train_crops.append(img)

    train_class_array = [1]
    train_class_desc = 'turbine'  
    with open('/home/ubuntu/data/sar/experiment_crops_20170815/train/distance_to_land/experiments_train_turbine.json') as json_data:
        json_train_data = json.load(json_data)
        for id_, item in json_train_data.iteritems():
            fn = id_.replace('.tif', '.png')
            train_filename.append(fn)
            train_feature.append(item['distance to land'])
            train_class.append(train_class_array)
            file_path = train_dir + '/' + train_class_desc + '/' + fn
            try:
                img  = imread(file_path)
            except IOError:
                continue
            train_crops.append(img)

    train_class_array = [0]
    train_class_desc = 'other'  
    with open('/home/ubuntu/data/sar/experiment_crops_20170815/train/distance_to_land/experiments_train_other.json') as json_data:
        json_train_data = json.load(json_data)
        for id_, item in json_train_data.iteritems():
            fn = id_.replace('.tif', '.png')
            train_filename.append(fn)
            train_feature.append(item['distance to land'])
            train_class.append(train_class_array)
            file_path = train_dir + '/' + train_class_desc + '/' + fn
            try:
                img  = imread(file_path)
            except IOError:
                continue
            train_crops.append(img)

    valid_class_array = [0]
    valid_class_desc = 'other'  
    with open('/home/ubuntu/data/sar/experiment_crops_20170815/validate/distance_to_land/experiments_validate_oil_and_gas_infrastructure.json') as json_data:
        json_validation_data = json.load(json_data)
        for id_, item in json_validation_data.iteritems():
            fn = id_.replace('.tif', '.png')
            valid_filename.append(fn)
            valid_feature.append(item['distance to land'])
            valid_class.append(valid_class_array)
            file_path = valid_dir + '/' + valid_class_desc + '/' + fn
            img  = imread(file_path)
            valid_crops.append(img)

    valid_class_array = [1]
    valid_class_desc = 'turbine'  
    with open('/home/ubuntu/data/sar/experiment_crops_20170815/validate/distance_to_land/experiments_validate_turbine.json') as json_data:
        json_validation_data = json.load(json_data)
        for id_, item in json_validation_data.iteritems():
            fn = id_.replace('.tif', '.png')
            valid_filename.append(fn)
            valid_feature.append(item['distance to land'])
            valid_class.append(valid_class_array)
            file_path = valid_dir + '/' + valid_class_desc + '/' + fn
            img  = imread(file_path)
            valid_crops.append(img)

    valid_class_array = [0]
    valid_class_desc = 'other'  
    with open('/home/ubuntu/data/sar/experiment_crops_20170815/validate/distance_to_land/experiments_validate_other.json') as json_data:
        json_validation_data = json.load(json_data)
        for id_, item in json_validation_data.iteritems():
            fn = id_.replace('.tif', '.png')
            valid_filename.append(fn)
            valid_feature.append(item['distance to land'])
            valid_class.append(valid_class_array)
            file_path = valid_dir + '/' + valid_class_desc + '/' + fn
            img  = imread(file_path)
            valid_crops.append(img)
 
    return train_crops, train_filename, train_feature, train_class, valid_crops, valid_filename, valid_feature, valid_class


In [4]:
input_shape = (50, 50, 1)
num_classes = 1

In [70]:
train_crops, train_filename, train_feature, train_class, \
valid_crops, valid_filename, valid_feature, valid_class = add_dist2land_experiment_crops_20170815_turbine()

In [71]:
print "training crops:", len(train_crops)
print "training features:", len(train_feature)
print "validation crops:", len(valid_crops)
print "validation features:", len(valid_feature)
print type(train_crops), type(train_crops[0]), train_crops[0].shape

training crops: 1996
training features: 1996
validation crops: 499
validation features: 499
<type 'list'> <type 'numpy.ndarray'> (50, 50)


In [80]:
# Reshape for keras format
np_train_class = np.array(train_class)
np_train_filename = np.array(train_filename)
np_train_crops = np.array(train_crops)
np_train_feature = np.array(train_feature)

np_valid_class = np.array(valid_class)
np_valid_filename = np.array(valid_filename)
np_valid_crops = np.array(valid_crops)
np_valid_feature = np.array(valid_feature)

np_train_crops = np.expand_dims(np_train_crops, axis=3)
np_train_feature = np.expand_dims(np_train_feature, axis=1)

np_valid_crops = np.expand_dims(np_valid_crops, axis=3)
np_valid_feature = np.expand_dims(np_valid_feature, axis=1)

In [81]:
print np_train_crops.shape
print np_train_feature.shape
print np_train_class.shape

print np_valid_crops.shape
print np_valid_feature.shape
print np_valid_class.shape

(1996, 50, 50, 1)
(1996, 1)
(1996, 1)
(499, 50, 50, 1)
(499, 1)
(499, 1)


# p = 0

In [82]:
p = 0
classifier_input = Input(shape=input_shape)
dist2land_input = Input(shape=(1,), name='dist2land_input')

x = Conv2D(32, (3, 3), activation='relu', padding='same')(classifier_input)
x = MaxPooling2D()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=-1)(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=-1)(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=-1)(x)
x = Conv2D(3, (3, 3), activation='relu', padding='same')(x)
x = Dropout(p)(x)

x = Flatten()(x)
x = merge([x, dist2land_input], 'concat')
x = Dense(num_classes, activation='sigmoid')(x)

model = Model(outputs=x, inputs=[classifier_input, dist2land_input])
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_7 (InputLayer)             (None, 50, 50, 1)     0                                            
____________________________________________________________________________________________________
conv2d_37 (Conv2D)               (None, 50, 50, 32)    320         input_7[0][0]                    
____________________________________________________________________________________________________
max_pooling2d_13 (MaxPooling2D)  (None, 25, 25, 32)    0           conv2d_37[0][0]                  
____________________________________________________________________________________________________
conv2d_38 (Conv2D)               (None, 25, 25, 64)    18496       max_pooling2d_13[0][0]           
___________________________________________________________________________________________

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:19: UserWarning:

The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.



In [84]:
lr = 0.001
K.set_value(model.optimizer.lr, lr)
model.fit([np_train_crops, np_train_feature], np_train_class,
          batch_size=32,
          epochs=10,
          validation_data=([np_valid_crops, np_valid_feature], np_valid_class))

Train on 1996 samples, validate on 499 samples
Epoch 1/10
1996/1996 [==============================] - 18s - loss: 0.5071 - acc: 0.8793 - val_loss: 0.2723 - val_acc: 0.9399
Epoch 2/10
1996/1996 [==============================] - 18s - loss: 0.2088 - acc: 0.9299 - val_loss: 0.2562 - val_acc: 0.9539
Epoch 3/10
1996/1996 [==============================] - 18s - loss: 0.1212 - acc: 0.9504 - val_loss: 0.1770 - val_acc: 0.9539
Epoch 4/10
1996/1996 [==============================] - 18s - loss: 0.1019 - acc: 0.9634 - val_loss: 0.1601 - val_acc: 0.9539
Epoch 5/10
1996/1996 [==============================] - 18s - loss: 0.0766 - acc: 0.9739 - val_loss: 0.1522 - val_acc: 0.9599
Epoch 6/10
1996/1996 [==============================] - 18s - loss: 0.0678 - acc: 0.9719 - val_loss: 0.1316 - val_acc: 0.9699
Epoch 7/10
1996/1996 [==============================] - 18s - loss: 0.0650 - acc: 0.9770 - val_loss: 0.1186 - val_acc: 0.9699
Epoch 8/10
1996/1996 [==============================] - 18s - loss: 0.0

In [85]:
lr = 0.0001
K.set_value(model.optimizer.lr, lr)
model.fit([np_train_crops, np_train_feature], np_train_class,
          batch_size=32,
          epochs=10,
          validation_data=([np_valid_crops, np_valid_feature], np_valid_class))

Train on 1996 samples, validate on 499 samples
Epoch 1/10
1996/1996 [==============================] - 18s - loss: 0.0193 - acc: 0.9945 - val_loss: 0.0886 - val_acc: 0.9800
Epoch 2/10
1996/1996 [==============================] - 18s - loss: 0.0047 - acc: 0.9990 - val_loss: 0.0889 - val_acc: 0.9780
Epoch 3/10
1996/1996 [==============================] - 18s - loss: 0.0030 - acc: 1.0000 - val_loss: 0.0887 - val_acc: 0.9780
Epoch 4/10
1996/1996 [==============================] - 18s - loss: 0.0018 - acc: 1.0000 - val_loss: 0.0915 - val_acc: 0.9820
Epoch 5/10
1996/1996 [==============================] - 18s - loss: 0.0012 - acc: 1.0000 - val_loss: 0.0928 - val_acc: 0.9800
Epoch 6/10
1996/1996 [==============================] - 18s - loss: 8.1095e-04 - acc: 1.0000 - val_loss: 0.0973 - val_acc: 0.9820
Epoch 7/10
1996/1996 [==============================] - 18s - loss: 4.8696e-04 - acc: 1.0000 - val_loss: 0.1002 - val_acc: 0.9820
Epoch 8/10
1996/1996 [==============================] - 18s - l

In [86]:
model.save('/home/ubuntu/data/sar/experiment_crops_20170815/binary_experiments/turbine_classifier/turbine_binary_distance_feature_model.h5')

# p = 0.3

In [87]:
p = 0.3
classifier_input = Input(shape=input_shape)
dist2land_input = Input(shape=(1,), name='dist2land_input')

x = Conv2D(32, (3, 3), activation='relu', padding='same')(classifier_input)
x = MaxPooling2D()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=-1)(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=-1)(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=-1)(x)
x = Conv2D(3, (3, 3), activation='relu', padding='same')(x)
x = Dropout(p)(x)

x = Flatten()(x)
x = merge([x, dist2land_input], 'concat')
x = Dense(num_classes, activation='sigmoid')(x)

model = Model(outputs=x, inputs=[classifier_input, dist2land_input])
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_8 (InputLayer)             (None, 50, 50, 1)     0                                            
____________________________________________________________________________________________________
conv2d_43 (Conv2D)               (None, 50, 50, 32)    320         input_8[0][0]                    
____________________________________________________________________________________________________
max_pooling2d_15 (MaxPooling2D)  (None, 25, 25, 32)    0           conv2d_43[0][0]                  
____________________________________________________________________________________________________
conv2d_44 (Conv2D)               (None, 25, 25, 64)    18496       max_pooling2d_15[0][0]           
___________________________________________________________________________________________

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:19: UserWarning:

The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.



In [88]:
lr = 0.001
K.set_value(model.optimizer.lr, lr)
model.fit([np_train_crops, np_train_feature], np_train_class,
          batch_size=32,
          epochs=10,
          validation_data=([np_valid_crops, np_valid_feature], np_valid_class))

Train on 1996 samples, validate on 499 samples
Epoch 1/10
1996/1996 [==============================] - 18s - loss: 0.3219 - acc: 0.8808 - val_loss: 0.1907 - val_acc: 0.9459
Epoch 2/10
1996/1996 [==============================] - 18s - loss: 0.1721 - acc: 0.9314 - val_loss: 1.8712 - val_acc: 0.5050
Epoch 3/10
1996/1996 [==============================] - 18s - loss: 0.1591 - acc: 0.9464 - val_loss: 0.1064 - val_acc: 0.9619
Epoch 4/10
1996/1996 [==============================] - 22s - loss: 0.1119 - acc: 0.9589 - val_loss: 0.1018 - val_acc: 0.9719
Epoch 5/10
1996/1996 [==============================] - 35s - loss: 0.0998 - acc: 0.9614 - val_loss: 0.0856 - val_acc: 0.9699
Epoch 6/10
1996/1996 [==============================] - 35s - loss: 0.0789 - acc: 0.9689 - val_loss: 0.1000 - val_acc: 0.9659
Epoch 7/10
1996/1996 [==============================] - 34s - loss: 0.0723 - acc: 0.9729 - val_loss: 0.2282 - val_acc: 0.9399
Epoch 8/10
1996/1996 [==============================] - 36s - loss: 0.0

In [89]:
lr = 0.0001
K.set_value(model.optimizer.lr, lr)
model.fit([np_train_crops, np_train_feature], np_train_class,
          batch_size=32,
          epochs=10,
          validation_data=([np_valid_crops, np_valid_feature], np_valid_class))

Train on 1996 samples, validate on 499 samples
Epoch 1/10
1996/1996 [==============================] - 34s - loss: 0.0315 - acc: 0.9875 - val_loss: 0.0756 - val_acc: 0.9780
Epoch 2/10
1996/1996 [==============================] - 35s - loss: 0.0144 - acc: 0.9960 - val_loss: 0.0694 - val_acc: 0.9800
Epoch 3/10
1996/1996 [==============================] - 35s - loss: 0.0089 - acc: 0.9985 - val_loss: 0.0722 - val_acc: 0.9820
Epoch 4/10
1996/1996 [==============================] - 35s - loss: 0.0067 - acc: 0.9985 - val_loss: 0.0733 - val_acc: 0.9820
Epoch 5/10
1996/1996 [==============================] - 35s - loss: 0.0037 - acc: 1.0000 - val_loss: 0.0786 - val_acc: 0.9820
Epoch 6/10
1996/1996 [==============================] - 34s - loss: 0.0038 - acc: 0.9995 - val_loss: 0.0838 - val_acc: 0.9820
Epoch 7/10
1996/1996 [==============================] - 35s - loss: 0.0046 - acc: 0.9985 - val_loss: 0.0852 - val_acc: 0.9800
Epoch 8/10
1996/1996 [==============================] - 35s - loss: 0.0

In [90]:
model.save('/home/ubuntu/data/sar/experiment_crops_20170815/binary_experiments/turbine_classifier/turbine_binary_distance_feature_dropout_p_0p3_model.h5')

# Binary other classifier, with distance-to-land feature

In [91]:
trained_model_dir = '/home/ubuntu/data/sar/experiment_crops_20170815/trained_models/5.0-as-binary_classifier/'
train_dir = '/home/ubuntu/data/sar/experiment_crops_20170815/binary_experiments/other_classifier/train/50x50'
valid_dir = '/home/ubuntu/data/sar/experiment_crops_20170815/binary_experiments/other_classifier/validate/50x50/'


In [92]:
input_shape = (50, 50, 1)
num_classes = 1

In [97]:
def add_dist2land_experiment_crops_20170815_other():
    
    train_dir = '/home/ubuntu/data/sar/experiment_crops_20170815/binary_experiments/other_classifier/train/50x50/'
    valid_dir = '/home/ubuntu/data/sar/experiment_crops_20170815/binary_experiments/other_classifier/validate/50x50/'

    train_class = []           
    train_filename = []
    train_crops = []
    train_feature = []

    valid_class = []
    valid_filename = []
    valid_crops = []
    valid_feature = []

    train_class_desc = 'oil_and_turbine'
    train_class_array = [0]
    with open('/home/ubuntu/data/sar/experiment_crops_20170815/train/distance_to_land/experiments_train_oil_and_gas_infrastructure.json') as json_data:
        json_train_data = json.load(json_data)
        for id_, item in json_train_data.iteritems():
            fn = id_.replace('.tif', '.png')
            train_filename.append(fn)
            train_feature.append(item['distance to land'])
            train_class.append(train_class_array)
            file_path = train_dir + '/' + train_class_desc + '/' + fn
            try:
                img  = imread(file_path)
            except IOError:
                continue
            train_crops.append(img)

    train_class_array = [0]
    train_class_desc = 'oil_and_turbine'  
    with open('/home/ubuntu/data/sar/experiment_crops_20170815/train/distance_to_land/experiments_train_turbine.json') as json_data:
        json_train_data = json.load(json_data)
        for id_, item in json_train_data.iteritems():
            fn = id_.replace('.tif', '.png')
            train_filename.append(fn)
            train_feature.append(item['distance to land'])
            train_class.append(train_class_array)
            file_path = train_dir + '/' + train_class_desc + '/' + fn
            try:
                img  = imread(file_path)
            except IOError:
                continue
            train_crops.append(img)

    train_class_array = [1]
    train_class_desc = 'other'  
    with open('/home/ubuntu/data/sar/experiment_crops_20170815/train/distance_to_land/experiments_train_other.json') as json_data:
        json_train_data = json.load(json_data)
        for id_, item in json_train_data.iteritems():
            fn = id_.replace('.tif', '.png')
            train_filename.append(fn)
            train_feature.append(item['distance to land'])
            train_class.append(train_class_array)
            file_path = train_dir + '/' + train_class_desc + '/' + fn
            try:
                img  = imread(file_path)
            except IOError:
                continue
            train_crops.append(img)

    valid_class_array = [0]
    valid_class_desc = 'oil_and_turbine'  
    with open('/home/ubuntu/data/sar/experiment_crops_20170815/validate/distance_to_land/experiments_validate_oil_and_gas_infrastructure.json') as json_data:
        json_validation_data = json.load(json_data)
        for id_, item in json_validation_data.iteritems():
            fn = id_.replace('.tif', '.png')
            valid_filename.append(fn)
            valid_feature.append(item['distance to land'])
            valid_class.append(valid_class_array)
            file_path = valid_dir + '/' + valid_class_desc + '/' + fn
            img  = imread(file_path)
            valid_crops.append(img)

    valid_class_array = [0]
    valid_class_desc = 'oil_and_turbine'  
    with open('/home/ubuntu/data/sar/experiment_crops_20170815/validate/distance_to_land/experiments_validate_turbine.json') as json_data:
        json_validation_data = json.load(json_data)
        for id_, item in json_validation_data.iteritems():
            fn = id_.replace('.tif', '.png')
            valid_filename.append(fn)
            valid_feature.append(item['distance to land'])
            valid_class.append(valid_class_array)
            file_path = valid_dir + '/' + valid_class_desc + '/' + fn
            img  = imread(file_path)
            valid_crops.append(img)

    valid_class_array = [1]
    valid_class_desc = 'other'  
    with open('/home/ubuntu/data/sar/experiment_crops_20170815/validate/distance_to_land/experiments_validate_other.json') as json_data:
        json_validation_data = json.load(json_data)
        for id_, item in json_validation_data.iteritems():
            fn = id_.replace('.tif', '.png')
            valid_filename.append(fn)
            valid_feature.append(item['distance to land'])
            valid_class.append(valid_class_array)
            file_path = valid_dir + '/' + valid_class_desc + '/' + fn
            img  = imread(file_path)
            valid_crops.append(img)
 
    return train_crops, train_filename, train_feature, train_class, valid_crops, valid_filename, valid_feature, valid_class



In [98]:
train_crops, train_filename, train_feature, train_class, \
valid_crops, valid_filename, valid_feature, valid_class = add_dist2land_experiment_crops_20170815_other()

In [99]:
print "training crops:", len(train_crops)
print "training features:", len(train_feature)
print "validation crops:", len(valid_crops)
print "validation features:", len(valid_feature)
print type(train_crops), type(train_crops[0]), train_crops[0].shape

training crops: 1996
training features: 1996
validation crops: 499
validation features: 499
<type 'list'> <type 'numpy.ndarray'> (50, 50)


In [100]:
# Reshape for keras format
np_train_class = np.array(train_class)
np_train_filename = np.array(train_filename)
np_train_crops = np.array(train_crops)
np_train_feature = np.array(train_feature)

np_valid_class = np.array(valid_class)
np_valid_filename = np.array(valid_filename)
np_valid_crops = np.array(valid_crops)
np_valid_feature = np.array(valid_feature)

np_train_crops = np.expand_dims(np_train_crops, axis=3)
np_train_feature = np.expand_dims(np_train_feature, axis=1)

np_valid_crops = np.expand_dims(np_valid_crops, axis=3)
np_valid_feature = np.expand_dims(np_valid_feature, axis=1)

In [101]:
print np_train_crops.shape
print np_train_feature.shape
print np_train_class.shape

print np_valid_crops.shape
print np_valid_feature.shape
print np_valid_class.shape

(1996, 50, 50, 1)
(1996, 1)
(1996, 1)
(499, 50, 50, 1)
(499, 1)
(499, 1)


# p = 0

Accuracy on validation stalls around 0.97

In [102]:
p = 0
classifier_input = Input(shape=input_shape)
dist2land_input = Input(shape=(1,), name='dist2land_input')

x = Conv2D(32, (3, 3), activation='relu', padding='same')(classifier_input)
x = MaxPooling2D()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=-1)(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=-1)(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=-1)(x)
x = Conv2D(3, (3, 3), activation='relu', padding='same')(x)
x = Dropout(p)(x)

x = Flatten()(x)
x = merge([x, dist2land_input], 'concat')
x = Dense(num_classes, activation='sigmoid')(x)

model = Model(outputs=x, inputs=[classifier_input, dist2land_input])
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_9 (InputLayer)             (None, 50, 50, 1)     0                                            
____________________________________________________________________________________________________
conv2d_49 (Conv2D)               (None, 50, 50, 32)    320         input_9[0][0]                    
____________________________________________________________________________________________________
max_pooling2d_17 (MaxPooling2D)  (None, 25, 25, 32)    0           conv2d_49[0][0]                  
____________________________________________________________________________________________________
conv2d_50 (Conv2D)               (None, 25, 25, 64)    18496       max_pooling2d_17[0][0]           
___________________________________________________________________________________________

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:19: UserWarning:

The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.



In [103]:
lr = 0.001
K.set_value(model.optimizer.lr, lr)
model.fit([np_train_crops, np_train_feature], np_train_class,
          batch_size=32,
          epochs=10,
          validation_data=([np_valid_crops, np_valid_feature], np_valid_class))

Train on 1996 samples, validate on 499 samples
Epoch 1/10
1996/1996 [==============================] - 35s - loss: 0.4324 - acc: 0.8377 - val_loss: 0.4751 - val_acc: 0.7735
Epoch 2/10
1996/1996 [==============================] - 34s - loss: 0.2515 - acc: 0.9073 - val_loss: 0.1810 - val_acc: 0.9419
Epoch 3/10
1996/1996 [==============================] - 35s - loss: 0.1565 - acc: 0.9404 - val_loss: 0.1848 - val_acc: 0.9238
Epoch 4/10
1996/1996 [==============================] - 35s - loss: 0.1239 - acc: 0.9514 - val_loss: 0.1492 - val_acc: 0.9499
Epoch 5/10
1996/1996 [==============================] - 35s - loss: 0.1121 - acc: 0.9574 - val_loss: 0.1735 - val_acc: 0.9379
Epoch 6/10
1996/1996 [==============================] - 35s - loss: 0.0782 - acc: 0.9694 - val_loss: 0.4967 - val_acc: 0.8377
Epoch 7/10
1996/1996 [==============================] - 35s - loss: 0.0742 - acc: 0.9734 - val_loss: 0.1384 - val_acc: 0.9579
Epoch 8/10
1996/1996 [==============================] - 35s - loss: 0.0

In [104]:
lr = 0.0001
K.set_value(model.optimizer.lr, lr)
model.fit([np_train_crops, np_train_feature], np_train_class,
          batch_size=32,
          epochs=10,
          validation_data=([np_valid_crops, np_valid_feature], np_valid_class))

Train on 1996 samples, validate on 499 samples
Epoch 1/10
1996/1996 [==============================] - 18s - loss: 0.0172 - acc: 0.9960 - val_loss: 0.1279 - val_acc: 0.9499
Epoch 2/10
1996/1996 [==============================] - 19s - loss: 0.0044 - acc: 0.9995 - val_loss: 0.1194 - val_acc: 0.9619
Epoch 3/10
1996/1996 [==============================] - 18s - loss: 0.0019 - acc: 1.0000 - val_loss: 0.1381 - val_acc: 0.9519
Epoch 4/10
1996/1996 [==============================] - 19s - loss: 0.0011 - acc: 1.0000 - val_loss: 0.1285 - val_acc: 0.9639
Epoch 5/10
1996/1996 [==============================] - 30s - loss: 6.0832e-04 - acc: 1.0000 - val_loss: 0.1258 - val_acc: 0.9619
Epoch 6/10
1996/1996 [==============================] - 34s - loss: 3.8086e-04 - acc: 1.0000 - val_loss: 0.1499 - val_acc: 0.9579
Epoch 7/10
1996/1996 [==============================] - 34s - loss: 2.0374e-04 - acc: 1.0000 - val_loss: 0.1615 - val_acc: 0.9579
Epoch 8/10
1996/1996 [==============================] - 34s

# p = 0.3

In [105]:
p = 0.3
classifier_input = Input(shape=input_shape)
dist2land_input = Input(shape=(1,), name='dist2land_input')

x = Conv2D(32, (3, 3), activation='relu', padding='same')(classifier_input)
x = MaxPooling2D()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=-1)(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=-1)(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=-1)(x)
x = Conv2D(3, (3, 3), activation='relu', padding='same')(x)
x = Dropout(p)(x)

x = Flatten()(x)
x = merge([x, dist2land_input], 'concat')
x = Dense(num_classes, activation='sigmoid')(x)

model = Model(outputs=x, inputs=[classifier_input, dist2land_input])
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_10 (InputLayer)            (None, 50, 50, 1)     0                                            
____________________________________________________________________________________________________
conv2d_55 (Conv2D)               (None, 50, 50, 32)    320         input_10[0][0]                   
____________________________________________________________________________________________________
max_pooling2d_19 (MaxPooling2D)  (None, 25, 25, 32)    0           conv2d_55[0][0]                  
____________________________________________________________________________________________________
conv2d_56 (Conv2D)               (None, 25, 25, 64)    18496       max_pooling2d_19[0][0]           
___________________________________________________________________________________________

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:19: UserWarning:

The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.



In [106]:
lr = 0.001
K.set_value(model.optimizer.lr, lr)
model.fit([np_train_crops, np_train_feature], np_train_class,
          batch_size=32,
          epochs=10,
          validation_data=([np_valid_crops, np_valid_feature], np_valid_class))

Train on 1996 samples, validate on 499 samples
Epoch 1/10
1996/1996 [==============================] - 33s - loss: 0.4501 - acc: 0.8101 - val_loss: 0.3883 - val_acc: 0.8236
Epoch 2/10
1996/1996 [==============================] - 32s - loss: 0.2956 - acc: 0.8848 - val_loss: 0.2846 - val_acc: 0.8818
Epoch 3/10
1996/1996 [==============================] - 33s - loss: 0.2269 - acc: 0.9083 - val_loss: 0.7400 - val_acc: 0.6453
Epoch 4/10
1996/1996 [==============================] - 33s - loss: 0.2051 - acc: 0.9228 - val_loss: 0.3110 - val_acc: 0.8537
Epoch 5/10
1996/1996 [==============================] - 33s - loss: 0.1731 - acc: 0.9394 - val_loss: 0.1576 - val_acc: 0.9399
Epoch 6/10
1996/1996 [==============================] - 33s - loss: 0.1475 - acc: 0.9444 - val_loss: 0.1333 - val_acc: 0.9359
Epoch 7/10
1996/1996 [==============================] - 33s - loss: 0.1222 - acc: 0.9529 - val_loss: 0.1140 - val_acc: 0.9599
Epoch 8/10
1996/1996 [==============================] - 32s - loss: 0.1

In [107]:
lr = 0.0001
K.set_value(model.optimizer.lr, lr)
model.fit([np_train_crops, np_train_feature], np_train_class,
          batch_size=32,
          epochs=10,
          validation_data=([np_valid_crops, np_valid_feature], np_valid_class))

Train on 1996 samples, validate on 499 samples
Epoch 1/10
1996/1996 [==============================] - 33s - loss: 0.0572 - acc: 0.9775 - val_loss: 0.1134 - val_acc: 0.9639
Epoch 2/10
1996/1996 [==============================] - 32s - loss: 0.0313 - acc: 0.9875 - val_loss: 0.1252 - val_acc: 0.9599
Epoch 3/10
1996/1996 [==============================] - 32s - loss: 0.0282 - acc: 0.9900 - val_loss: 0.1138 - val_acc: 0.9659
Epoch 4/10
1996/1996 [==============================] - 32s - loss: 0.0166 - acc: 0.9955 - val_loss: 0.1218 - val_acc: 0.9679
Epoch 5/10
1996/1996 [==============================] - 33s - loss: 0.0203 - acc: 0.9950 - val_loss: 0.1245 - val_acc: 0.9679
Epoch 6/10
1996/1996 [==============================] - 32s - loss: 0.0163 - acc: 0.9945 - val_loss: 0.1114 - val_acc: 0.9699
Epoch 7/10
1996/1996 [==============================] - 32s - loss: 0.0106 - acc: 0.9980 - val_loss: 0.1134 - val_acc: 0.9719
Epoch 8/10
1996/1996 [==============================] - 30s - loss: 0.0

In [108]:
lr = 0.0001
K.set_value(model.optimizer.lr, lr)
model.fit([np_train_crops, np_train_feature], np_train_class,
          batch_size=32,
          epochs=10,
          validation_data=([np_valid_crops, np_valid_feature], np_valid_class))

Train on 1996 samples, validate on 499 samples
Epoch 1/10
1996/1996 [==============================] - 18s - loss: 0.0068 - acc: 0.9990 - val_loss: 0.1299 - val_acc: 0.9699
Epoch 2/10
1996/1996 [==============================] - 18s - loss: 0.0067 - acc: 0.9985 - val_loss: 0.1321 - val_acc: 0.9639
Epoch 3/10
1996/1996 [==============================] - 18s - loss: 0.0043 - acc: 1.0000 - val_loss: 0.1268 - val_acc: 0.9719
Epoch 4/10
1996/1996 [==============================] - 18s - loss: 0.0056 - acc: 0.9980 - val_loss: 0.1232 - val_acc: 0.9679
Epoch 5/10
1996/1996 [==============================] - 18s - loss: 0.0045 - acc: 0.9995 - val_loss: 0.1264 - val_acc: 0.9699
Epoch 6/10
1996/1996 [==============================] - 18s - loss: 0.0043 - acc: 0.9995 - val_loss: 0.1344 - val_acc: 0.9619
Epoch 7/10
1996/1996 [==============================] - 18s - loss: 0.0040 - acc: 0.9995 - val_loss: 0.1201 - val_acc: 0.9699
Epoch 8/10
1996/1996 [==============================] - 18s - loss: 0.0

In [109]:
model.save('/home/ubuntu/data/sar/experiment_crops_20170815/binary_experiments/turbine_classifier/other_binary_distance_feature_dropout_p_0p3_model.h5')

# Binary oil and gas classifier, with distance-to-land feature

In [5]:
trained_model_dir = '/home/ubuntu/data/sar/experiment_crops_20170815/trained_models/5.0-as-binary_classifier/'
train_dir = '/home/ubuntu/data/sar/experiment_crops_20170815/binary_experiments/oil_and_gas_infrastructure_classifier/train/50x50'
valid_dir = '/home/ubuntu/data/sar/experiment_crops_20170815/binary_experiments/oil_and_gas_infrastructure_classifier/validate/50x50/'

In [6]:
input_shape = (50, 50, 1)
num_classes = 1

In [12]:
def add_dist2land_experiment_crops_20170815_oil():
    
    train_dir = '/home/ubuntu/data/sar/experiment_crops_20170815/binary_experiments/oil_and_gas_infrastructure_classifier/train/50x50/'
    valid_dir = '/home/ubuntu/data/sar/experiment_crops_20170815/binary_experiments/oil_and_gas_infrastructure_classifier/validate/50x50/'

    train_class = []           
    train_filename = []
    train_crops = []
    train_feature = []

    valid_class = []
    valid_filename = []
    valid_crops = []
    valid_feature = []

    train_class_desc = 'oil_and_gas_infrastructure'
    train_class_array = [1]
    with open('/home/ubuntu/data/sar/experiment_crops_20170815/train/distance_to_land/experiments_train_oil_and_gas_infrastructure.json') as json_data:
        json_train_data = json.load(json_data)
        for id_, item in json_train_data.iteritems():
            fn = id_.replace('.tif', '.png')
            train_filename.append(fn)
            train_feature.append(item['distance to land'])
            train_class.append(train_class_array)
            file_path = train_dir + '/' + train_class_desc + '/' + fn
            img  = imread(file_path)
            train_crops.append(img)

    train_class_array = [0]
    train_class_desc = 'other'  
    with open('/home/ubuntu/data/sar/experiment_crops_20170815/train/distance_to_land/experiments_train_turbine.json') as json_data:
        json_train_data = json.load(json_data)
        for id_, item in json_train_data.iteritems():
            fn = id_.replace('.tif', '.png')
            train_filename.append(fn)
            train_feature.append(item['distance to land'])
            train_class.append(train_class_array)
            file_path = train_dir + '/' + train_class_desc + '/' + fn
            img  = imread(file_path)
            train_crops.append(img)

    train_class_array = [0]
    train_class_desc = 'other'  
    with open('/home/ubuntu/data/sar/experiment_crops_20170815/train/distance_to_land/experiments_train_other.json') as json_data:
        json_train_data = json.load(json_data)
        for id_, item in json_train_data.iteritems():
            fn = id_.replace('.tif', '.png')
            train_filename.append(fn)
            train_feature.append(item['distance to land'])
            train_class.append(train_class_array)
            file_path = train_dir + '/' + train_class_desc + '/' + fn
            img  = imread(file_path)
            train_crops.append(img)

    valid_class_array = [1]
    valid_class_desc = 'oil_and_gas_infrastructure'  
    with open('/home/ubuntu/data/sar/experiment_crops_20170815/validate/distance_to_land/experiments_validate_oil_and_gas_infrastructure.json') as json_data:
        json_validation_data = json.load(json_data)
        for id_, item in json_validation_data.iteritems():
            fn = id_.replace('.tif', '.png')
            valid_filename.append(fn)
            valid_feature.append(item['distance to land'])
            valid_class.append(valid_class_array)
            file_path = valid_dir + '/' + valid_class_desc + '/' + fn
            img  = imread(file_path)
            valid_crops.append(img)

    valid_class_array = [0]
    valid_class_desc = 'other'  
    with open('/home/ubuntu/data/sar/experiment_crops_20170815/validate/distance_to_land/experiments_validate_turbine.json') as json_data:
        json_validation_data = json.load(json_data)
        for id_, item in json_validation_data.iteritems():
            fn = id_.replace('.tif', '.png')
            valid_filename.append(fn)
            valid_feature.append(item['distance to land'])
            valid_class.append(valid_class_array)
            file_path = valid_dir + '/' + valid_class_desc + '/' + fn
            img  = imread(file_path)
            valid_crops.append(img)

    valid_class_array = [0]
    valid_class_desc = 'other'  
    with open('/home/ubuntu/data/sar/experiment_crops_20170815/validate/distance_to_land/experiments_validate_other.json') as json_data:
        json_validation_data = json.load(json_data)
        for id_, item in json_validation_data.iteritems():
            fn = id_.replace('.tif', '.png')
            valid_filename.append(fn)
            valid_feature.append(item['distance to land'])
            valid_class.append(valid_class_array)
            file_path = valid_dir + '/' + valid_class_desc + '/' + fn
            img  = imread(file_path)
            valid_crops.append(img)
 
    return train_crops, train_filename, train_feature, train_class, valid_crops, valid_filename, valid_feature, valid_class



In [13]:
train_crops, train_filename, train_feature, train_class, \
valid_crops, valid_filename, valid_feature, valid_class = add_dist2land_experiment_crops_20170815_oil()

In [14]:
print "training crops:", len(train_crops)
print "training features:", len(train_feature)
print "validation crops:", len(valid_crops)
print "validation features:", len(valid_feature)
print type(train_crops), type(train_crops[0]), train_crops[0].shape

training crops: 1996
training features: 1996
validation crops: 499
validation features: 499
<type 'list'> <type 'numpy.ndarray'> (50, 50)


In [15]:
# Reshape for keras format
np_train_class = np.array(train_class)
np_train_filename = np.array(train_filename)
np_train_crops = np.array(train_crops)
np_train_feature = np.array(train_feature)

np_valid_class = np.array(valid_class)
np_valid_filename = np.array(valid_filename)
np_valid_crops = np.array(valid_crops)
np_valid_feature = np.array(valid_feature)

np_train_crops = np.expand_dims(np_train_crops, axis=3)
np_train_feature = np.expand_dims(np_train_feature, axis=1)

np_valid_crops = np.expand_dims(np_valid_crops, axis=3)
np_valid_feature = np.expand_dims(np_valid_feature, axis=1)

In [16]:
print np_train_crops.shape
print np_train_feature.shape
print np_train_class.shape

print np_valid_crops.shape
print np_valid_feature.shape
print np_valid_class.shape

(1996, 50, 50, 1)
(1996, 1)
(1996, 1)
(499, 50, 50, 1)
(499, 1)
(499, 1)


# p = 0.3

In [17]:
p = 0.3
classifier_input = Input(shape=input_shape)
dist2land_input = Input(shape=(1,), name='dist2land_input')

x = Conv2D(32, (3, 3), activation='relu', padding='same')(classifier_input)
x = MaxPooling2D()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=-1)(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=-1)(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=-1)(x)
x = Conv2D(3, (3, 3), activation='relu', padding='same')(x)
x = Dropout(p)(x)

x = Flatten()(x)
x = merge([x, dist2land_input], 'concat')
x = Dense(num_classes, activation='sigmoid')(x)

model = Model(outputs=x, inputs=[classifier_input, dist2land_input])
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 50, 50, 1)     0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 50, 50, 32)    320         input_1[0][0]                    
____________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)   (None, 25, 25, 32)    0           conv2d_1[0][0]                   
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 25, 25, 64)    18496       max_pooling2d_1[0][0]            
___________________________________________________________________________________________

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:19: UserWarning:

The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.

/usr/local/lib/python2.7/dist-packages/keras/legacy/layers.py:460: UserWarning:

The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.



In [18]:
lr = 0.001
K.set_value(model.optimizer.lr, lr)
model.fit([np_train_crops, np_train_feature], np_train_class,
          batch_size=32,
          epochs=10,
          validation_data=([np_valid_crops, np_valid_feature], np_valid_class))

Train on 1996 samples, validate on 499 samples
Epoch 1/10
1996/1996 [==============================] - 18s - loss: 0.7732 - acc: 0.8126 - val_loss: 0.4563 - val_acc: 0.8377
Epoch 2/10
1996/1996 [==============================] - 18s - loss: 0.1710 - acc: 0.9409 - val_loss: 0.1162 - val_acc: 0.9619
Epoch 3/10
1996/1996 [==============================] - 18s - loss: 0.1066 - acc: 0.9589 - val_loss: 0.9482 - val_acc: 0.6192
Epoch 4/10
1996/1996 [==============================] - 18s - loss: 0.0809 - acc: 0.9714 - val_loss: 0.1176 - val_acc: 0.9559
Epoch 5/10
1996/1996 [==============================] - 18s - loss: 0.0714 - acc: 0.9744 - val_loss: 0.0834 - val_acc: 0.9739
Epoch 6/10
1996/1996 [==============================] - 18s - loss: 0.0656 - acc: 0.9760 - val_loss: 0.0845 - val_acc: 0.9739
Epoch 7/10
1996/1996 [==============================] - 18s - loss: 0.0575 - acc: 0.9805 - val_loss: 0.1067 - val_acc: 0.9599
Epoch 8/10
1996/1996 [==============================] - 18s - loss: 0.0

In [19]:
lr = 0.001
K.set_value(model.optimizer.lr, lr)
model.fit([np_train_crops, np_train_feature], np_train_class,
          batch_size=32,
          epochs=5,
          validation_data=([np_valid_crops, np_valid_feature], np_valid_class))

Train on 1996 samples, validate on 499 samples
Epoch 1/5
1996/1996 [==============================] - 18s - loss: 0.0307 - acc: 0.9870 - val_loss: 0.1558 - val_acc: 0.9579
Epoch 2/5
1996/1996 [==============================] - 18s - loss: 0.0264 - acc: 0.9880 - val_loss: 0.1269 - val_acc: 0.9679
Epoch 3/5
1996/1996 [==============================] - 18s - loss: 0.0318 - acc: 0.9890 - val_loss: 0.1332 - val_acc: 0.9699
Epoch 4/5
1996/1996 [==============================] - 18s - loss: 0.0203 - acc: 0.9950 - val_loss: 0.2665 - val_acc: 0.9579
Epoch 5/5
1996/1996 [==============================] - 18s - loss: 0.0253 - acc: 0.9920 - val_loss: 0.1263 - val_acc: 0.9699


In [20]:
lr = 0.0001
K.set_value(model.optimizer.lr, lr)
model.fit([np_train_crops, np_train_feature], np_train_class,
          batch_size=32,
          epochs=5,
          validation_data=([np_valid_crops, np_valid_feature], np_valid_class))

Train on 1996 samples, validate on 499 samples
Epoch 1/5
1996/1996 [==============================] - 18s - loss: 0.0079 - acc: 0.9985 - val_loss: 0.1139 - val_acc: 0.9719
Epoch 2/5
1996/1996 [==============================] - 18s - loss: 0.0060 - acc: 0.9985 - val_loss: 0.1135 - val_acc: 0.9719
Epoch 3/5
1996/1996 [==============================] - 18s - loss: 0.0075 - acc: 0.9970 - val_loss: 0.1429 - val_acc: 0.9719
Epoch 4/5
1996/1996 [==============================] - 18s - loss: 0.0064 - acc: 0.9975 - val_loss: 0.1326 - val_acc: 0.9699
Epoch 5/5
1996/1996 [==============================] - 18s - loss: 0.0029 - acc: 0.9990 - val_loss: 0.1435 - val_acc: 0.9699


In [21]:
lr = 0.0001
K.set_value(model.optimizer.lr, lr)
model.fit([np_train_crops, np_train_feature], np_train_class,
          batch_size=32,
          epochs=5,
          validation_data=([np_valid_crops, np_valid_feature], np_valid_class))

Train on 1996 samples, validate on 499 samples
Epoch 1/5
1996/1996 [==============================] - 18s - loss: 0.0021 - acc: 0.9995 - val_loss: 0.1295 - val_acc: 0.9699
Epoch 2/5
1996/1996 [==============================] - 18s - loss: 0.0020 - acc: 1.0000 - val_loss: 0.1246 - val_acc: 0.9719
Epoch 3/5
1996/1996 [==============================] - 18s - loss: 0.0018 - acc: 0.9995 - val_loss: 0.1321 - val_acc: 0.9699
Epoch 4/5
1996/1996 [==============================] - 18s - loss: 0.0015 - acc: 0.9995 - val_loss: 0.1329 - val_acc: 0.9699
Epoch 5/5
1996/1996 [==============================] - 18s - loss: 0.0018 - acc: 0.9995 - val_loss: 0.1502 - val_acc: 0.9699


In [23]:
model.save('/home/ubuntu/data/sar/experiment_crops_20170815/binary_experiments/oil_and_gas_infrastructure_classifier/oil_binary_distance_feature_dropout_p_0p3_model.h5')